In [2]:
## Income Classification with PySpark

# Environment Setup
from pyspark.sql import SparkSession
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F
import pandas as pd
import numpy as np

# Start Spark session
spark = SparkSession.builder.appName('ml-income').getOrCreate()

# Load data
df = spark.read.csv('/income.csv', header=True, nullValue='?',
                    ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True, inferSchema=True)

# Show schema and sample
df.printSchema()
df.show(5)

# Drop missing values
df = df.na.drop()

# Transpose view (via Pandas)
pd.DataFrame(df.take(5), columns=df.columns).transpose()

# Descriptive stats
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas().transpose()

# Encode categorical columns
label_cols = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "citizenship"]
label_out = ["label_" + col for col in label_cols]
indexer = StringIndexer(inputCols=label_cols, outputCols=label_out)
df = indexer.fit(df).transform(df)

# Assemble features
assembler = VectorAssembler(inputCols=label_out, outputCol="features")
df = assembler.transform(df)

# Encode target
label_indexer = StringIndexer(inputCol="income_class", outputCol="label")
df = label_indexer.fit(df).transform(df)

# Train-test split
train, test = df.randomSplit([0.7, 0.3])

# Random Forest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", maxBins=41)
model_rf = rf.fit(train)
pred_rf = model_rf.transform(test)
pred_rf.select("features", "label", "prediction").show(5)

# Decision Tree
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxBins=41)
model_dt = dt.fit(train)
pred_dt = model_dt.transform(test)
pred_dt.select("features", "label", "prediction").show(5)

# Evaluation function
def evaluate_model(predictions, name="Model"):
    print(f"\n{name} Evaluation:")
    for m in ["accuracy", "weightedPrecision", "weightedRecall", "f1"]:
        score = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName=m).evaluate(predictions)
        print(f"{m.capitalize()}: {score:.4f}")
    print(f"Test Error: {1.0 - score:.4f}")
    preds = predictions.select("prediction", "label").withColumn("label", F.col("label").cast(FloatType()))
    metrics = MulticlassMetrics(preds.rdd.map(tuple))
    print("Confusion Matrix:")
    print(metrics.confusionMatrix().toArray())

# Evaluate both models
evaluate_model(pred_rf, "Random Forest")
evaluate_model(pred_dt, "Decision Tree")

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education_years: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: integer (nullable = true)
 |-- capital_loss: integer (nullable = true)
 |-- hours_per_week: integer (nullable = true)
 |-- citizenship: string (nullable = true)
 |-- income_class: string (nullable = true)

+---+----------------+------+---------+---------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+-------------+------------+
|age|       workclass|weight|education|education_years|    marital_status|       occupation| relationship| race|   sex|capital_gain|capital_loss|hours_per_week|  citizenship|

/usr/local/lib/python3.11/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Confusion Matrix:
[[6235.  453.]
 [1191. 1042.]]

Decision Tree Evaluation:
Accuracy: 0.8124
Weightedprecision: 0.8038
Weightedrecall: 0.8124
F1: 0.8064
Test Error: 0.1936
Confusion Matrix:
[[6037.  651.]
 [1023. 1210.]]
